In [ ]:
import ulmo
import pandas as pd
import datetime
import future, sys
import psycopg2


_username = 'postgres'
_password = '123456'
_hostname = 'localhost'
_database = 'WeatherDB2' #### !!!!


In [ ]:
EU_countries = []
with open('F:\\NOAA\\EU.txt') as f:
    lines = f.readlines()
    for line in lines:
        EU_countries.append(line.split('\t')[0].strip())
# EU_countries

countries = { }
with open('F:\\NOAA\\ghcnd-countries.txt') as f:
    lines = f.readlines()
    for line in lines:
        line = line.replace("\n", "").strip()
        if line == "": 
            continue
        countries[line[3:]] = line[:2]
target_countries = []
for x in EU_countries:
    target_countries.append(countries[x])
#@a

target_countries += [ "KS", "JA", "CH", "TU", "RS"] #"US",
#target_countries = [ "JA" ]
print len(target_countries)

In [ ]:
def sum_dicts(d1, d2):
    d = { }
    for k in d1:
        if k == 'count':
            continue
        if k.startswith('max'):
            d[k] = max(d1[k], d2[k])
        elif k.startswith('min'):
            d[k] = min(d1[k], d2[k])
        else:
            d[k] = d1[k] + d2[k]
    if 'count' not in d:
        d['count'] = 0
    d['count'] += 1
    return d

def find_avg(d):
    for k in d:
        if k == 'count':
            continue
        if not k.startswith('max') and not k.startswith('min'):
            d[k] = float(d[k]) / d['count']
    return d        

def new_result():
    return {'dew_point': 0.0,
     'max_temp': -1000000.0,
     'max_wind_speed': -1000000.0,
     'mean_temp': 0.0,
     'mean_wind_speed': 0.0,
     'min_temp': 1000000.0, 
     'visibility': 0.0,
     'station_pressure': 0.0,
     'sea_level_pressure': 0.0,
     'max_gust': -1000000.0}

In [ ]:
bad_countries = []
bad_stations = []

stations = [ ]
d = { }
skiped = 0
with psycopg2.connect(host=_hostname, user=_username, password=_password, database=_database) as conn:
    with conn.cursor() as cur:
        #add country
        for country in target_countries:
            cur.execute("SELECT id FROM countries WHERE name=%s", [country])
            if cur.fetchone() == None:
                cur.execute("INSERT INTO countries (name) VALUES (%s) RETURNING id", [country])
                country_id = cur.fetchone()[0]
            else:
                continue
                
            try:
                stations = ulmo.ncdc.gsod.get_stations(country=country)
            except:
                bad_countries.append(country)
                sys.stdout.write("BaD country!")
                sys.stdout.flush()
            if len(stations) == 0:
                continue
                
            results = { }
    
#             for s in stations:
            try:
                d = ulmo.ncdc.gsod.get_data(stations.keys())
            except Exception, e:
                msg = country + "," + str(e)
                bad_stations.append(msg)
                sys.stdout.write(msg)
                sys.stdout.flush()

            for station, data in d.iteritems():
                if data == None:
                    skiped += 1
                    continue

                for row in data:
                    if row['date'] not in results:
                        results[row['date']] = new_result()
                    results[row['date']] = sum_dicts(results[row['date']], row)

            sys.stdout.write(country + "_downloaded ")
            sys.stdout.flush()

            for date in results:
                results[date] = find_avg(results[date])
                row = results[date]
                cur.execute("INSERT INTO data ("+
                                "sea_level_pressure, max_gust, max_wind_speed, dew_point, "+
                                "min_temp, mean_temp, station_pressure, mean_wind_speed, "+
                                "visibility, max_temp"+
                                ", country_id) \
                                VALUES(%s, %s, %s, %s," +
                                      "%s, %s, %s, %s," +
                                      "%s, %s," +
                                      "%s)", 
                                [row['sea_level_pressure'], row['max_gust'], row['max_wind_speed'], row['dew_point'], 
                                 row['min_temp'], row['mean_temp'], row['station_pressure'], row['mean_wind_speed'], 
                                 row['visibility'], row['max_temp'],
                                 country_id])
            conn.commit()
            sys.stdout.write(country + "_saved ")
            sys.stdout.flush()
            del results

In [ ]:
", ".join(['snow_depth', 'mean_temp_count', 'USAF', 'max_gust', 'min_temp_flag', 'dew_point_count', 'min_temp', 'precip_flag', 'mean_wind_speed', 'visibility_count', 'max_temp_flag', 'sea_level_pressure', 'mean_wind_speed_count', 'FRSHTT', 'visibility', 'max_wind_speed', 'date', 'sea_level_pressure_count', 'dew_point', 'mean_temp', 'station_pressure_count', 'precip', 'WBAN', 'station_pressure', 'max_temp', 'station_id'])

In [ ]:
country = "GM"
stations = ulmo.ncdc.gsod.get_stations(country=country)

In [ ]:
d = ulmo.ncdc.gsod.get_data(stations.keys()[:2])

In [ ]:
for station, data in d.iteritems():
    print station
    print type(data)

In [ ]:
# for x in table_fields.split(", "):
#     print row[]
s = ', '.join(['%s'] * len(table_fields.split(", "))) + ", %s"
s = "( " + s + " )"
params = table_fields + ", station_id"
p = params.split(', ')
# print params
print p

In [ ]:
(d[d.keys()[0]])[0]

In [ ]:
# class Country:
#     def __init__(self, name, fullname):
#         self.name = name
#         self.fullname = fullname
#         self.day = Day()

# class Day:
#     def __init__(self, date):
#         self.weather = Weather()
    
# class Weather:
#     def __init__(self, attributes):
#         self.attributes = attributes